In [1]:
import os
%matplotlib inline

import os
from collections import Counter, OrderedDict
import numpy as np
from operator import itemgetter
import matplotlib.pyplot as plt
from astropy.table import Table
import multiprocessing
from cesium.time_series import TimeSeries
import cesium.featurize as featurize
from tqdm import tnrange, tqdm_notebook, tqdm
import sklearn 
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import seaborn as sns
import pandas as pd

In [2]:
### Read data files train_meta and train
os.chdir('/Users/shengyao/Downloads/Kaggle Data')

In [64]:
train_meta_data = pd.read_csv('training_set_metadata.csv')
train = pd.read_csv('training_set.csv')
train['mjd'] = train['mjd'].apply(int)

In [67]:
df_list = list()
columns_label = ['object_id','mjd']
for i in range(6):
    new = ['flux'+str(i),'flux_err'+str(i),'detected'+str(i)]
    columns_label = columns_label + new


In [68]:
grouped = train.groupby(['object_id','mjd'])
for groups in tqdm(grouped):
    entry = pd.DataFrame(data=[[None]*20],columns=columns_label)
    entry['object_id'] = int(groups[0][0])
    entry['mjd'] = int(groups[0][1])

    for i in range(6):
        if i in groups[1]['passband'].values:
            entry['flux'+str(i)] = groups[1][groups[1]['passband']==i]['flux'].values[0]
            entry['flux_err'+str(i)] = groups[1][groups[1]['passband']==i]['flux_err'].values[0]
            entry['detected'+str(i)] = groups[1][groups[1]['passband']==i]['detected'].values[0]
        if i not in groups[1]['passband'].values:
            entry['flux'+str(i)] = np.nan
            entry['flux_err'+str(i)] = np.nan
            entry['detected'+str(i)] = np.nan
    df_list.append(entry)
    del entry

100%|██████████| 17667/17667 [04:24<00:00, 68.32it/s]


In [69]:
output_df = pd.concat(df_list)